In [6]:
import pandas as pd
import openpyxl
from openpyxl.styles import Alignment, numbers

#月次excelデータ
file = "瀬戸工場生産Ｇがいし.xlsx"

#奉行から出力した月次決算excelデータ
data = "Book1.xlsx"

#奉行から出力した補助科目集計表excelデータの読み込み
data2 = "Book2.xlsx"

def seto_gaishi(file,data,data2):
    #######################################################
    #各種データの読み込み
    #######################################################
    
    #月次excelデータ
    wb = openpyxl.load_workbook(filename=file, read_only=False)
    ws = wb.worksheets[1]
    
    #奉行から出力した月次決算excelデータ
    xlsx = pd.ExcelFile(data)
    sheet_names = xlsx.sheet_names
    
    #奉行から出力した補助科目集計表excelデータの読み込み
    df2 = pd.read_excel(data2)
    
    #######################################################
    #読み込んだデータの整形
    #######################################################
    
    #奉行月次決算Excelデータ内の複数シートを整形
    #結合
    df = pd.concat(
        pd.read_excel(xlsx,sheet_name=sheet)
            for sheet in sheet_names)
    
    # インデックスリセット
    df.reset_index(drop=True, inplace=True)
    df = df.fillna(0)
    df = df.replace('\u3000', '', regex=True)
    
    #奉行補助科目集計表excelデータ内の複数シートを整形
    # インデックスをリセット
    df2.reset_index(drop=True, inplace=True)
    df2 = df2.fillna(0)
    df2 = df2.replace('\u3000', '', regex=True)
    
    #######################################################
    #補助科目集計表詳細データの集計
    #######################################################
    
    #global変数の定義
    supplies_plan = 0
    supplies_sudden = 0
    supplies_normal = 0
    supplies_grant = 0
    repair_plan = 0
    repair_sudden = 0
    repair_normal = 0
    
    #消耗品、修繕費（通常、計画、突発）の集計(forで回せると思うが、今後編集)
    for sub1 in range(len(df2.axes[0])):
        if df2.iloc[sub1,2] == "5470":
            if "1" in str(df2.iloc[sub1,4]):
                supplies_plan = supplies_plan + df2.iloc[sub1,7] - df2.iloc[sub1,8]
            elif "2" in str(df2.iloc[sub1,4]):
                supplies_sudden = supplies_sudden + df2.iloc[sub1,7] - df2.iloc[sub1,8]
            elif "0" in str(df2.iloc[sub1,4]):
                supplies_normal = supplies_normal + df2.iloc[sub1,7] - df2.iloc[sub1,8]
        elif df2.iloc[sub1,2] == "5471":
            if "1" in str(df2.iloc[sub1,4]):
                supplies_plan = supplies_plan + df2.iloc[sub1,7] - df2.iloc[sub1,8]
            elif "2" in str(df2.iloc[sub1,4]):
                supplies_sudden = supplies_sudden + df2.iloc[sub1,7] - df2.iloc[sub1,8]
            elif "0" in str(df2.iloc[sub1,4]):
                supplies_normal = supplies_normal + df2.iloc[sub1,7] - df2.iloc[sub1,8]
            elif "4" in str(df2.iloc[sub1,4]):
                supplies_grant = supplies_grant + df2.iloc[sub1,7] - df2.iloc[sub1,8]
        elif df2.iloc[sub1,2] == "5480":
            if "1" in str(df2.iloc[sub1,4]):
                repair_plan = repair_plan + df2.iloc[sub1,7] - df2.iloc[sub1,8]
            elif "2" in str(df2.iloc[sub1,4]):
                repair_sudden = repair_sudden + df2.iloc[sub1,7] - df2.iloc[sub1,8]
            elif "0" in str(df2.iloc[sub1,4]):
                repair_normal = repair_normal + df2.iloc[sub1,7] - df2.iloc[sub1,8]
        elif df2.iloc[sub1,2] == "5481":
            if "1" in str(df2.iloc[sub1,4]):
                repair_plan = repair_plan + df2.iloc[sub1,7] - df2.iloc[sub1,8]
            elif "2" in str(df2.iloc[sub1,4]):
                repair_sudden = repair_sudden + df2.iloc[sub1,7] - df2.iloc[sub1,8]
            elif "0" in str(df2.iloc[sub1,4]):
                repair_normal = repair_normal + df2.iloc[sub1,7] - df2.iloc[sub1,8]
    
    #各補助科目データのまるめ処理
    supplies_plan_round = round(supplies_plan / 1000)
    supplies_sudden_round = round(supplies_sudden / 1000)
    supplies_normal_round = round(supplies_normal / 1000)
    supplies_grant_round = round(supplies_grant / 1000)
    repair_plan_round = round(repair_plan / 1000)
    repair_sudden_round = round(repair_sudden / 1000)
    repair_normal_round = round(repair_normal / 1000)
    
    account_names = []
    
    #補助科目データフレームの作成
    df_kamoku = pd.DataFrame({
        "大科目":
        ["消耗品費","消耗品費","消耗品費","消耗品費","修繕費","修繕費","修繕費"],
        "科目":
        ["消耗品費（通常）","消耗品費（計画）","消耗品費（突発）","消耗品費（補助金）","修繕費（通常）","修繕費（計画）","修繕費（突発）"],
        "変数":
        [supplies_normal,supplies_plan,supplies_sudden,supplies_grant,repair_normal,repair_plan,repair_sudden],
        "変数まるめ":
        [supplies_normal_round,supplies_plan_round,supplies_sudden_round,supplies_grant_round,repair_normal_round,repair_plan_round,repair_sudden_round]
    })
    
    #下3桁の算出
    df_kamoku["変数下3桁"] = df_kamoku["変数"].apply(lambda x: int(str(x)[-3:]))
    #数値0を除外
    df_kamoku = df_kamoku[df_kamoku["変数下3桁"] != 0]
    
    #合計値判定用
    #奉行月次決算データと同等
    supplies_total = round((supplies_plan + supplies_sudden + supplies_normal)/1000)
    repair_total = round((repair_plan + repair_sudden + repair_normal)/1000)
    #値ごとにまるめているため差異発生可能性がある
    supplies_total1 = supplies_plan_round + supplies_sudden_round + supplies_normal_round
    repair_total1 = repair_plan_round + repair_sudden_round + repair_normal_round
    
    #上記合計値の集計表データフレームを作成
    df_total = pd.DataFrame({
        "科目":
        ["消耗品","修繕費"],
        "奉行":
        [supplies_total,repair_total],
        "まるめ":
        [supplies_total1,repair_total1]
    })
    
    
    #######################################################
    #メインプログラム
    #######################################################
    
    df_month = pd.DataFrame({
        "月":
        ["1月","2月","3月","4月","5月","6月","7月","8月","9月","10月","11月","12月"],
        "翌月":
        ["2月","3月","4月","5月","6月","7月","8月","9月","10月","11月","12月","1月"]
    })
    
    #月次Excelの更新
    for i in range(len(df_month.axes[0])):
        if df_month.iloc[i,0] == ws.cell(1,6).value :
            ws.cell(1,6).value = df_month.iloc[i,1]
            break
    
    #月次Excel(B column)の参照
    for i in range(ws.max_row):
        #奉行データ(df.axes[0]縦方向)の参照
        for j in range(len(df.axes[0])):
            #月次Excelのセル値と奉行データフレームの値が合致してれば以下の処理(for col～)
            if ws.cell(i+1,2).value == df.iloc[j,1]:
                #月次Excel t～AE(1月~3月)とcell(1,6)を参照し、t~AEの列に数値を入れ込む
                for col in range(20,ws.max_column):
                    if ws.cell(3,col).value == ws.cell(1 , 6).value : 
                        ws.cell(i+1,col).value = df.iloc[j,2]
                        #数値の整形[カンマ、小数点なし、右詰め]
                        ws.cell(i+1,col).number_format = "#,##0"
                        ws.cell(i+1,col).alignment = Alignment(horizontal='right')
                        break
    
    
    #瀬戸がいしのみ適用
    x = 0
    y = 0
    for i in range(len(df.axes[0])):
        #奉行データ内に"5221(内部仕入 堺工場 がいしG)"があるかを確認
        if df.iloc[i,0] == "5221":
            #5221内部仕入高
            x = df.iloc[i,2]
            #5210仕入高
            y = df.iloc[i-1,2]
            #月次Excel内の"材料仕入高"に合致した行を探す
            for j in range(ws.max_row):
                if ws.cell(j+1,2).value == "材料仕入高":
                    #4月～3月の間で、最新の値を上書きする
                    for col in range(20,ws.max_column):
                        if ws.cell(3,col).value == ws.cell(1 , 6).value : 
                            ws.cell(j+1,col).value = x+y
                            #数値の整形[カンマ、小数点なし、右詰め]
                            ws.cell(j+1,col).number_format = "#,##0"
                            ws.cell(j+1,col).alignment = Alignment(horizontal='right')
                            break
    
    
    #補助科目内訳表の転記
    #月次Excel(B column)の参照
    for i in range(ws.max_row):
        #奉行データ(df.axes[0]縦方向)の参照
        for j in range(len(df_kamoku.axes[0])):
            #月次Excelのセル値と奉行データフレームの値が合致してれば以下の処理(for col～)
            if ws.cell(i+1,2).value == df_kamoku.iloc[j,1]:
                #補助科目内訳表内容の比較
                for k in range(len(df_total.axes[0])):
                    #月次Excel t～AE(1月~3月)とcell(1,6)を参照し、t~AEの列に数値を入れ込む
                    for col in range(20,ws.max_column):
                        if ws.cell(3,col).value == ws.cell(1 , 6).value : 
                            ws.cell(i+1,col).value = df_kamoku.iloc[j,3]
                            #数値の整形[カンマ、小数点なし、右詰め]
                            ws.cell(i+1,col).number_format = "#,##0"
                            ws.cell(i+1,col).alignment = Alignment(horizontal='right')
                            break
                            
    #補助科目内訳表の合計値比較
    df_min_or_max = {}
    zz = 0
    #月次Excel(B column)の参照
    for i in range(len(df_total.axes[0])):
        #奉行補助科目合計値が個別まるめ合計値よりも小さい場合の処理
        if df_total.iloc[i,1] < df_total.iloc[i,2]:
            #補助科目集計表データフレームの縦軸方向に処理
            for j in range(len(df_kamoku.axes[0])):
                if df_total.iloc[i,0] == df_kamoku.iloc[j,0]:
                    df_min_or_max[df_kamoku.iloc[j,1]] = df_kamoku.iloc[j,4]
            min_value = min(df_min_or_max.values())
            min_key = [k for k, v in df_min_or_max.items() if v == min_value][0]
            for i in range(len(df_kamoku.axes[0])):
                if df_kamoku.iloc[i,4] == min_value:
                    zz = df_kamoku.iloc[i,3]-1
        #奉行補助科目合計値が個別まるめ合計値よりも大きい場合の処理
        elif df_total.iloc[i,1] > df_total.iloc[i,2]:
            for j in range(len(df_kamoku.axes[0])):
                if df_total.iloc[i,0] == df_kamoku.iloc[j,0]:
                    df_min_or_max[df_kamoku.iloc[j,1]] = df_kamoku.iloc[j,4]
            max_value = min(df_min_or_max.values())
            max_key = [k for k, v in df_min_or_max.items() if v == max_value][0]
            for i in range(len(df_kamoku.axes[0])):
                if df_kamoku.iloc[i,4] == max_value:
                    zz = df_kamoku.iloc[i,3]+1
    
    #上記判定により差異が判明した場合、変数zzに格納されている数値を上書き
    for i in range(ws.max_row):
        for j in range(len(df_kamoku.axes[0])):
            if ws.cell(i+1,2).value == df_kamoku.iloc[j,1]:
                #月次Excel t～AE(1月~3月)とcell(1,6)を参照し、t~AEの列に数値を入れ込む
                for col in range(20,ws.max_column):
                    if ws.cell(3,col).value == ws.cell(1 , 6).value : 
                        if ws.cell(i+1,2).value == min_key:
                            ws.cell(i+1,col).value = zz
                            #数値の整形[カンマ、小数点なし、右詰め]
                            ws.cell(i+1,col).number_format = "#,##0"
                            ws.cell(i+1,col).alignment = Alignment(horizontal='right')
                            break
    return wb

wb = seto_gaishi(file,data,data2)
wb.save("瀬戸工場生産Ｇがいし2.xlsx")